In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

from scipy.stats import zscore

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
df = pd.read_excel('./data/default of credit card clients.xlsx', header=1)  
df.rename(columns={'default payment next month': 'DEFAULT'}, inplace=True)

In [ ]:
print("Dataset Shape:", df.shape)
print("\n" + "="*50)
print("First 5 Rows:")
print(df.head())

print("\n"+"="*50)
print("Missing Values:")
print(df.isnull().sum())

print("\n" + "="*50)
print("Target Variable Distribution:")
print(df['DEFAULT'].value_counts(normalize=True))

In [ ]:
print(f"Duplicate IDs: {df['ID'].duplicated().sum()}")

print("\nGender Distribution:")
print(df['SEX'].value_counts())

print("\nEducation Distribution:")
print(df['EDUCATION'].value_counts())

print("\nMarriage Distribution:")
print(df['MARRIAGE'].value_counts())

In [ ]:
df['EDUCATION'] = df['EDUCATION'].replace({0:4, 5:4, 6:4})

df['MARRIAGE'] = df['EDUCATION'].replace({0:3})

In [ ]:
#Age Disto
plt.figure(figsize=(14,5))

plt.subplot(1, 3, 1)
sns.histplot(df['AGE'], bins=30, kde=True, color='skyblue')
plt.title("Age Distribution")
plt.xlabel("Age")

plt.subplot(1, 3, 2)
sns.countplot(x='SEX', hue="DEFAULT", data=df, palette='Set2')
plt.title("Default Rate by Gender")
plt.xlabel("Gender (1=Male, 2=Female)")
plt.legend(title="Default", labels=['No', 'Yes'])

plt.subplot(1, 3, 3)
sns.countplot(x='EDUCATION', hue='DEFAULT', data=df, palette='Set1')
plt.title('Default Rate by Education')
plt.xlabel('Education Level')
plt.legend(title='Default', labels=['No', 'Yes'])

plt.tight_layout()
plt.show()

In [ ]:
#Credit limit Distro
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
sns.histplot(df['LIMIT_BAL'], bins=50, kde=True, color='green')
plt.title('Credit Limit Distribution')
plt.xlabel('Credit Limit (NT$)')

plt.subplot(1, 2, 2)
sns.boxplot(x='DEFAULT', y='LIMIT_BAL', data=df, palette='coolwarm')
plt.title('Credit Limit by Default Status')
plt.xlabel('Default (0=No, 1=Yes)')
plt.ylabel('Credit Limit (NT$)')

plt.tight_layout()
plt.show()

In [ ]:
print("Credit Limit Statistics by Default Status:")
print(df.groupby('DEFAULT')['LIMIT_BAL'].describe())

In [ ]:
# Recent payment status (PAY_0)
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.countplot(x='PAY_0', hue='DEFAULT', data=df, palette='viridis')
plt.title('Most Recent Payment Status vs Default')
plt.xlabel('PAY_0 (-1=On time, 1=1mo delay, 2=2mo delay...)')
plt.legend(title='Default', labels=['No', 'Yes'])

plt.subplot(1, 2, 2)
# Default rate by payment status
default_by_pay = df.groupby('PAY_0')['DEFAULT'].mean().sort_index()
default_by_pay.plot(kind='bar', color='coral')
plt.title('Default Rate by Recent Payment Status')
plt.xlabel('PAY_0')
plt.ylabel('Default Rate')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

print("\nDefault Rate by PAY_0:")
print(default_by_pay)

In [ ]:
#Correlation Analysis
correlation_features = ['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_0', 'PAY_2', 'PAY_3', 'DEFAULT']

corr_matrix = df[correlation_features].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap="RdYlGn_r", center=0, square=True, linewidths=1)
plt.title('Correlation Matrix: Key Risk Factors', fontsize=16)
plt.tight_layout()
plt.show()

# Print strongest correlations with DEFAULT
print("\nStrongest Correlations with DEFAULT:")
default_corr = corr_matrix['DEFAULT'].sort_values(ascending=False)
print(default_corr)

**Key Insight to Note:** `PAY_0`, `PAY_2`, `PAY_3` (recent payment delays) show strongest positive correlation with defaults.

In [ ]:
# Generate comprehensive summary
print("="*70)
print("DATASET SUMMARY STATISTICS")
print("="*70)

print(df.describe())

print("\n" + "="*70)
print("DEFAULT RATE BY KEY DEMOGRAPHICS")
print("="*70)

print("\nBy Gender:")
print(df.groupby('SEX')['DEFAULT'].agg(['mean', 'count']))

print("\nBy Education:")
print(df.groupby('EDUCATION')['DEFAULT'].agg(['mean', 'count']))

print("\nBy Age Group:")
df['AGE_GROUP'] = pd.cut(df['AGE'], bins=[0, 30, 40, 50, 100], 
                         labels=['18-30', '31-40', '41-50', '50+'])
print(df.groupby('AGE_GROUP')['DEFAULT'].agg(['mean', 'count']))

In [ ]:
#Credit Util Features
#Average bill util (spending as % of credit limit)
bill_cols= ['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']
df['AVG_BILL'] = df[bill_cols].mean(axis=1)
df['AVG_BILL_UTIL'] = (df['AVG_BILL'] / df['LIMIT_BAL']).replace([np.inf, -np.inf], 0)

#Cap util at 200% (some may overpend limit)
df['AVG_BILL_UTIL'] = df['AVG_BILL_UTIL'].clip(upper=2.0)

#Maximum bill util (highest spedning month)
df['MAX_BILL_UTIL'] = (df[bill_cols].max(axis=1) / df['LIMIT_BAL']).replace([np.inf, -np.inf], 0)
df['MAX_BILL_UTIL'] = df['MAX_BILL_UTIL'].clip(upper=2.0)

In [ ]:
#Payment Behavior Metrics
pay_cols = ['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']
df['AVG_PAYMENT'] = df[pay_cols].mean(axis=1)

df['PAY_CONSISTENCY'] = df[pay_cols].std(axis=1)

df['PAY_TO_BILL_RATIO'] = (df['AVG_PAYMENT'] / df['AVG_BILL']).replace([np.inf, -np.inf], 0)
df['PAY_TO_BILL_RATIO'] = df['PAY_TO_BILL_RATIO'].clip(upper=2.0)


In [ ]:
#Delinquency Features
#average pay delay across 6 months
pay_status_cols = ['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
df['AVG_PAY_STATUS'] = df[pay_status_cols].mean(axis=1)

#recent delay trend(postive = worsening, negative=improving)
df['RECENT_DELAY_TREND'] = df['PAY_0'] - df['PAY_3']

#pay > 0
df['NUM_DELAYS'] = (df[pay_status_cols] > 0).sum(axis=1)

# Max delay severity
df['MAX_DELAY'] = df[pay_status_cols].max(axis=1)

In [ ]:
#Total outstanding debt
df['TOTAL_DEBT'] = df[bill_cols].sum(axis=1)

# Debt to limit ratio
df['DEBT_TO_LIMIT'] = (df['TOTAL_DEBT'] / (df['LIMIT_BAL'] * 6)).replace([np.inf, -np.inf], 0)
df['DEBT_TO_LIMIT'] = df['DEBT_TO_LIMIT'].clip(upper=2.0)


In [ ]:
#Validate New features 
new_features = ['AVG_BILL_UTIL', 'MAX_BILL_UTIL', 'AVG_PAYMENT', 'PAY_CONSISTENCY','PAY_TO_BILL_RATIO', 'AVG_PAY_STATUS', 'RECENT_DELAY_TREND', 'NUM_DELAYS', 'MAX_DELAY', 'TOTAL_DEBT', 'DEBT_TO_LIMIT']

print(df[new_features].describe())

In [ ]:
# Visualize engineered features vs DEFAULT
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

key_features = ['AVG_BILL_UTIL', 'PAY_TO_BILL_RATIO', 'AVG_PAY_STATUS', 'NUM_DELAYS', 'MAX_DELAY', 'DEBT_TO_LIMIT']

for idx, feature in enumerate(key_features):
    sns.boxplot(x='DEFAULT', y=feature, data=df, ax=axes[idx], palette='Set2')
    axes[idx].set_title(f'{feature} by Default Status')
    axes[idx].set_xlabel('Default (0=No, 1=Yes)')

plt.tight_layout()
plt.show()


In [ ]:
print("\nNew Features Correlation with DEFAULT:")
feature_corr = df[new_features + ['DEFAULT']].corr()['DEFAULT'].sort_values(ascending=False)
print(feature_corr)

In [ ]:
print("="*80)
print("OUTLIER DETECTION & TREATMENT")
print("="*80)

# Visualize outliers before treatment
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

sample_features = ['LIMIT_BAL', 'AGE', 'AVG_BILL_UTIL', 'NUM_DELAYS', 'TOTAL_DEBT', 'MAX_DELAY']

for idx, feature in enumerate(sample_features):
    if idx < len(axes):
        axes[idx].boxplot(X[feature], vert=True)
        axes[idx].set_title(f'{feature} - Original')
        axes[idx].set_ylabel('Value')

plt.tight_layout()
plt.show()

# IQR Method (Industry Standard)
def remove_outliers_iqr(df, columns, threshold=3.0):
    """Remove outliers using IQR method"""
    mask = pd.Series([True] * len(df))
    
    outlier_counts = {}
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        
        col_mask = (df[col] >= lower_bound) & (df[col] <= upper_bound)
        outliers_in_col = (~col_mask).sum()
        outlier_counts[col] = outliers_in_col
        
        mask = mask & col_mask
    
    print("\nOutliers detected per feature:")
    for col, count in sorted(outlier_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"  {col}: {count} ({count/len(df)*100:.1f}%)")
    
    return mask

# Apply outlier removal
outlier_mask = remove_outliers_iqr(X, clustering_features, threshold=3.0)

X_clean = X[outlier_mask].copy()
df_clean = df[outlier_mask].copy()

print(f"\n{'='*80}")
print(f"Removed {len(X) - len(X_clean)} rows with extreme outliers ({((len(X) - len(X_clean))/len(X)*100):.1f}%)")
print(f"Clean dataset shape: {X_clean.shape}")
print(f"Retention rate: {len(X_clean)/len(X)*100:.1f}%")
print(f"{'='*80}")

# Visualize after treatment
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for idx, feature in enumerate(sample_features):
    if idx < len(axes):
        axes[idx].boxplot(X_clean[feature], vert=True)
        axes[idx].set_title(f'{feature} - After Outlier Removal')
        axes[idx].set_ylabel('Value')

plt.tight_layout()
plt.show()

In [ ]:
#Standardise Features 
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clean)


In [ ]:
# Calculate inertia for k=1 to k=10
inertias = []
silhouette_scores = []
K_range = range(2, 11)

print("Testing cluster counts from 2 to 10...")

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plot Elbow Curve
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(K_range, inertias, marker='o', linewidth=2, markersize=8, color='navy')
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Inertia (Within-Cluster Sum of Squares)', fontsize=12)
ax1.set_title('Elbow Method: Finding Optimal k', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.axvline(x=4, color='red', linestyle='--', label='Suggested k=4', alpha=0.7)
ax1.legend()

# Plot Silhouette Scores
ax2.plot(K_range, silhouette_scores, marker='s', linewidth=2, markersize=8, color='darkgreen')
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score by Cluster Count', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.axvline(x=4, color='red', linestyle='--', label='Suggested k=4', alpha=0.7)
ax2.axhline(y=0.3, color='orange', linestyle=':', label='Good threshold (>0.3)', alpha=0.7)
ax2.legend()

plt.tight_layout()
plt.show()

print("\nSilhouette Scores:")
for k, score in zip(K_range, silhouette_scores):
    print(f"k={k}: {score:.3f}")

In [ ]:
optimal_k = 4

kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=20, max_iter=300)
df_clean['SEGMENT'] = kmeans_final.fit_predict(X_scaled)

final_silhouette = silhouette_score(X_scaled, df_clean['SEGMENT'])
print(f"\n✓ K-Means clustering complete with k={optimal_k}")
print(f"Final Silhouette Score: {final_silhouette:.3f}")


In [ ]:
#Analyse Segments 
segment_profile = df_clean.groupby('SEGMENT').agg({
    'ID': 'count',
    'LIMIT_BAL': 'mean',
    'AGE': 'mean',
    'AVG_BILL_UTIL': 'mean',
    'PAY_TO_BILL_RATIO': 'mean',
    'AVG_PAY_STATUS': 'mean',
    'NUM_DELAYS': 'mean',
    'MAX_DELAY': 'mean',
    'DEFAULT': 'mean'
}).round(2)

segment_profile.columns = ['Count', 'Avg_Credit_Limit', 'Avg_Age', 
                           'Avg_Utilization', 'Pay_to_Bill_Ratio', 
                           'Avg_Pay_Status', 'Avg_Num_Delays', 'Max_Delay', 
                           'Default_Rate']

print("\n" + "="*80)
print("CUSTOMER SEGMENT PROFILES")
print("="*80)
print(segment_profile)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Segment sizes
axes[0, 0].bar(segment_profile.index, segment_profile['Count'], color='steelblue')
axes[0, 0].set_title('Segment Sizes', fontsize=14)
axes[0, 0].set_xlabel('Segment')
axes[0, 0].set_ylabel('Number of Customers')

# Default rates
colors = ['green', 'yellow', 'orange', 'red']
axes[0, 1].bar(segment_profile.index, segment_profile['Default_Rate'], 
               color=colors, alpha=0.7)
axes[0, 1].set_title('Default Rate by Segment', fontsize=14)
axes[0, 1].set_xlabel('Segment')
axes[0, 1].set_ylabel('Default Rate')
axes[0, 1].axhline(y=0.22, color='black', linestyle='--', label='Overall Avg', alpha=0.5)
axes[0, 1].legend()

# Credit limits
axes[1, 0].bar(segment_profile.index, segment_profile['Avg_Credit_Limit'], color='purple', alpha=0.6)
axes[1, 0].set_title('Average Credit Limit by Segment', fontsize=14)
axes[1, 0].set_xlabel('Segment')
axes[1, 0].set_ylabel('Credit Limit (NT$)')

# Payment delays
axes[1, 1].bar(segment_profile.index, segment_profile['Avg_Num_Delays'], color='coral')
axes[1, 1].set_title('Average Number of Delays by Segment', fontsize=14)
axes[1, 1].set_xlabel('Segment')
axes[1, 1].set_ylabel('Avg Delays (6 months)')

plt.tight_layout()
plt.show()

In [ ]:
segment_names = {}

# Analyze each segment to assign names
for seg in range(optimal_k):
    profile = segment_profile.loc[seg]
    
    if profile['Default_Rate'] < 0.15 and profile['Avg_Credit_Limit'] > 150000:
        segment_names[seg] = 'Premium Low-Risk'
    elif profile['Default_Rate'] > 0.30:
        segment_names[seg] = 'High-Risk Delinquents'
    elif profile['Avg_Utilization'] > 0.6:
        segment_names[seg] = 'High-Utilizers'
    else:
        segment_names[seg] = 'Stable Mid-Tier'

# Apply names
df_clean['SEGMENT_NAME'] = df_clean['SEGMENT'].map(segment_names)

print("\n" + "="*80)
print("SEGMENT NAMES:")
print("="*80)
for seg, name in segment_names.items():
    count = (df_clean['SEGMENT'] == seg).sum()
    default_rate = df_clean[df_clean['SEGMENT'] == seg]['DEFAULT'].mean()
    print(f"Segment {seg}: {name} (n={count}, default_rate={default_rate:.1%})")



In [ ]:
#Risk Exploration 
# Detailed default analysis
print("="*80)
print("DEFAULT ANALYSIS BY SEGMENT")
print("="*80)

for seg in range(optimal_k):
    seg_data = df_clean[df_clean['SEGMENT'] == seg]
    seg_name = segment_names[seg]
    
    print(f"\n{seg_name} (Segment {seg}):")
    print(f"  Total Customers: {len(seg_data)}")
    print(f"  Defaults: {seg_data['DEFAULT'].sum()}")
    print(f"  Default Rate: {seg_data['DEFAULT'].mean():.1%}")
    print(f"  Avg Credit Limit: NT${seg_data['LIMIT_BAL'].mean():,.0f}")
    print(f"  Avg Age: {seg_data['AGE'].mean():.1f} years")
    print(f"  Avg Utilization: {seg_data['AVG_BILL_UTIL'].mean():.1%}")
    print(f"  Avg Payment Delays: {seg_data['NUM_DELAYS'].mean():.1f}")

In [ ]:
#Risk Factor Correlations Within High-Risk Segment
high_risk_seg = segment_profile['Default_Rate'].idxmax()
high_risk_data = df_clean[df_clean['SEGMENT'] == high_risk_seg]

print(f"\n{'='*80}")
print(f"RISK DRIVERS IN {segment_names[high_risk_seg].upper()}")
print(f"{'='*80}")

# Correlation with DEFAULT in high-risk segment
risk_features = ['LIMIT_BAL', 'AGE', 'AVG_BILL_UTIL', 'PAY_TO_BILL_RATIO',
                 'AVG_PAY_STATUS', 'NUM_DELAYS', 'MAX_DELAY', 'DEFAULT']

risk_corr = high_risk_data[risk_features].corr()['DEFAULT'].sort_values(ascending=False)
print("\nTop Risk Drivers (Correlation with Default):")
print(risk_corr)

# Visualize
plt.figure(figsize=(10, 6))
risk_corr[:-1].plot(kind='barh', color='crimson', alpha=0.7)
plt.title(f'Risk Factor Correlations in {segment_names[high_risk_seg]}', fontsize=14)
plt.xlabel('Correlation with Default')
plt.axvline(x=0, color='black', linewidth=0.8)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix

print("\n" + "="*80)
print("BONUS: PREDICTIVE MODELING (Logistic Regression)")
print("="*80)

# Prepare data
X_model = df_clean[clustering_features]
y_model = df_clean['DEFAULT']

X_train, X_test, y_train, y_test = train_test_split(
    X_model, y_model, test_size=0.3, random_state=42, stratify=y_model
)

# Scale
scaler_model = StandardScaler()
X_train_scaled = scaler_model.fit_transform(X_train)
X_test_scaled = scaler_model.transform(X_test)

# Train model
log_reg = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
log_reg.fit(X_train_scaled, y_train)

# Predictions
y_pred = log_reg.predict(X_test_scaled)
y_pred_proba = log_reg.predict_proba(X_test_scaled)[:, 1]

# Evaluate
auc_score = roc_auc_score(y_test, y_pred_proba)
print(f"\nROC-AUC Score: {auc_score:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['No Default', 'Default']))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(7, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['No Default', 'Default'],
            yticklabels=['No Default', 'Default'])
plt.title('Confusion Matrix - Default Prediction')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': clustering_features,
    'Coefficient': log_reg.coef_[0]
}).sort_values('Coefficient', ascending=False)

print("\nTop Predictive Features:")
print(feature_importance)


In [ ]:
output_path = 'data/credit_data_with_segments.csv'
df_clean.to_csv(output_path, index=False)

print(f"\n✓ Data exported to: {output_path}")
print(f"  Total records: {len(df_clean)}")
print(f"  Total columns: {len(df_clean.columns)}")
